# EMT Simulation of CIGRE MV without PV Inverters

### Run simulation

In [ ]:
import requests
import glob

def download_grid_data(name, url):
    with open(name, 'wb') as out_file:
        content = requests.get(url, stream=True).content
        out_file.write(content)

url = 'https://raw.githubusercontent.com/dpsim-simulator/cim-grid-data/master/CIGRE_MV/NEPLAN/CIGRE_MV_no_tapchanger_noLoad1_LeftFeeder_With_LoadFlow_Results/Rootnet_FULL_NE_28J17h'
filename = 'CIGRE-MV'
download_grid_data(filename+'_EQ.xml', url+'_EQ.xml')
download_grid_data(filename+'_TP.xml', url+'_TP.xml')
download_grid_data(filename+'_SV.xml', url+'_SV.xml')

files = glob.glob(filename+'_*.xml')
print(files)

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import matplotlib.pyplot as plt
import dpsimpy

In [ ]:
time_step = 1e-3
final_time = 1

In [ ]:
sim_name = 'EMT_CIGRE_MV_withoutDG'
sim_name_pf = sim_name + "_Powerflow"

### Powerflow for Initialization

In [ ]:
dpsimpy.Logger.set_log_dir('logs/' + sim_name_pf)
reader = dpsimpy.CIMReader(sim_name_pf, dpsimpy.LogLevel.debug, dpsimpy.LogLevel.debug)
system_pf = reader.loadCIM(50, files, dpsimpy.Domain.SP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.PVNode)

logger_pf = dpsimpy.Logger(sim_name_pf)
for node in system_pf.nodes:
    logger_pf.log_attribute(node.name() + '.V', 'v', node)

sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(1.0)
sim_pf.set_final_time(2.0)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.set_solver_component_behaviour(dpsimpy.SolverBehaviour.Initialization)
sim_pf.do_init_from_nodes_and_terminals(True)

sim_pf.add_logger(logger_pf)
sim_pf.run()


### Dynamic simulation

In [ ]:
dpsimpy.Logger.set_log_dir('logs/' + sim_name)
reader2 = dpsimpy.CIMReader(sim_name, dpsimpy.LogLevel.info, dpsimpy.LogLevel.debug)
system_emt = reader2.loadCIM(50, files, dpsimpy.Domain.EMT, dpsimpy.PhaseType.ABC, dpsimpy.GeneratorType.NONE)

system_emt.init_with_powerflow(system_pf)

# log node voltages
logger_emt= dpsimpy.Logger(sim_name)
for node in system_emt.nodes:
    logger_emt.log_attribute(node.name() + '.V', 'v', node)

# log line and load currents
for comp in system_emt.components:
    if isinstance(comp, dpsimpy.emt.ph3.PiLine):
        logger_emt.log_attribute(comp.name() + '.I', 'i_intf', comp)
    if isinstance(comp, dpsimpy.emt.ph3.RXLoad):
        logger_emt.log_attribute(comp.name() + '.I', 'i_intf', comp)

sim_emt = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.debug)
sim_emt.set_system(system_emt)
sim_emt.set_time_step(time_step)
sim_emt.set_final_time(final_time)
sim_emt.set_domain(dpsimpy.Domain.EMT)
sim_emt.set_solver(dpsimpy.Solver.MNA)
sim_emt.do_steady_state_init(False)
sim_emt.do_init_from_nodes_and_terminals(True)

sim_emt.add_logger(logger_emt)
sim_emt.run()

## Powerflow

### Read results

In [ ]:
modelName = 'EMT_CIGRE_MV_withoutDG'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim_powerflow = read_timeseries_csv(dpsim_result_file)

### Plot results

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim_powerflow.items():
    if ts_name != 'N0.V':
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.legend()
plt.show()

## EMT

### Read results

In [ ]:
modelName = 'EMT_CIGRE_MV_withoutDG'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim = read_timeseries_csv(dpsim_result_file)

### Plot Node voltages

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-3:] == 'V_0' and ts_name[:-2] != 'N0.V':
        plt.plot(ts_obj.time, ts_obj.values, label=ts_name)
        plt.plot(ts_dpsim_powerflow[ts_name].time, np.sqrt(2/3)*ts_dpsim_powerflow[ts_name].abs().values, label=ts_name)
plt.xlim([0,0.1])
plt.legend()
plt.show()

### At first timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-4:] == '.V_0':
        print(ts_name + ': ' + str(ts_obj.values[0]))

### At last timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-4:] == '.V_0':
        print(ts_name + ': ' + str(ts_obj.values[-1:]))

## Load currents

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-4:] == '.I_0' and ts_name[:4]=='LOAD':
        plt.plot(ts_obj.time, ts_obj.values, label=ts_name)
#plt.xlim([0,0.1])
plt.legend(loc='upper right')
plt.show()

### At first timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-4:] == '.I_0' and ts_name[:4]=='LOAD':
        print(ts_name + ': ' + str(ts_obj.values[0]))

### At last timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-4:] == '.I_0' and ts_name[:4]=='LOAD':
        print(ts_name + ': ' + str(ts_obj.values[-1]))

### Load Power Calculations

In [ ]:
# load_I_3_p ref from CIM: P=225250, Q=139597
# load_H_11_p ref from CIM: P=329800, Q=82656

load_I_3_p = ts_dpsim['N3.V_0'].values*ts_dpsim['LOAD-I-3.I_0'].values+ts_dpsim['N3.V_1'].values*ts_dpsim['LOAD-I-3.I_1'].values+ts_dpsim['N3.V_2'].values*ts_dpsim['LOAD-I-3.I_2'].values
load_H_11_p = ts_dpsim['N11.V_0'].values*ts_dpsim['LOAD-H-11.I_0'].values+ts_dpsim['N11.V_1'].values*ts_dpsim['LOAD-H-11.I_1'].values+ts_dpsim['N11.V_2'].values*ts_dpsim['LOAD-H-11.I_2'].values

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(ts_dpsim['N3.V_0'].time, load_I_3_p, label='load_I_3_p')
plt.plot(ts_dpsim['N11.V_0'].time, load_H_11_p, label='load_H_11_p')
#plt.xlim([0,0.01])
plt.legend(loc='upper right')
plt.show()

### At first timestep

In [ ]:
print('Load-I-3.p' + ': ' + str(load_I_3_p[0]))
print('Load-H-11.p' + ': ' + str(load_H_11_p[0]))

### At last timestep

In [ ]:
print('Load-I-3.p' + ': ' + str(load_I_3_p[-1]))
print('Load-H-11.p' + ': ' + str(load_H_11_p[-1]))

## Line currents

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-4:] == '.I_0' and ts_name[:4]!='LOAD':
        plt.plot(ts_obj.time, ts_obj.values, label=ts_name)
#plt.xlim([0,0.1])
plt.legend(loc='upper right')
plt.show()

### At first timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-4:] == '.I_0' and ts_name[:4]!='LOAD':
        print(ts_name + ': ' + str(ts_obj.values[0]))

### At last timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-4:] == '.I_0' and ts_name[:4]!='LOAD':
        print(ts_name + ': ' + str(ts_obj.values[-1]))